In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from pprint import pprint
import json
import random
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from tqdm import trange, tqdm
from sklearn.model_selection import train_test_split

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
X_AUX_PATH = '/Users/reza/School/2025/1-Spring/Advanced ML/Project/data/new/preprocessed_selected_features/train/aux.csv'
X_FNC_PATH = '/Users/reza/School/2025/1-Spring/Advanced ML/Project/data/new/preprocessed_selected_features/train/connectome_matrices.csv'
Y_PATH = '/Users/reza/School/2025/1-Spring/Advanced ML/Project/data/new/preprocessed_selected_features/train/labels.csv'

In [4]:
X_fnc_df = pd.read_csv(X_FNC_PATH)
X_aux_df = pd.read_csv(X_AUX_PATH)
y_df = pd.read_csv(Y_PATH)
X_fnc_df.set_index('participant_id', inplace=True)
X_aux_df.set_index('participant_id', inplace=True)
y_df.set_index('participant_id', inplace=True)
X_aux_df = X_aux_df.reindex(X_fnc_df.index)
y_df = y_df.reindex(X_fnc_df.index)

In [5]:
X_fnc_df.head()

,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,0throw_10thcolumn,...,195throw_196thcolumn,195throw_197thcolumn,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn
participant_id,,,,,,,,,,,,,,,,,,,,,
70z8Q2xdTXM3,0.270399,0.580746,0.485892,0.064059,0.617299,0.337467,0.550230,-0.087434,0.579197,0.535364,...,0.305246,0.583610,0.540900,0.228036,0.343643,0.485184,0.019701,0.614717,0.524420,0.404290
WHWymJu6zNZi,0.745668,0.635297,0.560712,0.541223,0.439375,0.473556,0.129684,-0.095509,0.132499,0.314387,...,0.295154,-0.021363,0.000563,-0.119118,0.511165,0.396962,0.201877,0.664817,0.612853,0.557002
4PAQp1M6EyAo,-0.141711,0.503933,0.294476,0.697041,0.840358,0.476623,0.690517,0.215647,0.573338,0.430980,...,0.464666,-0.031043,-0.048386,0.092712,0.464166,0.337855,0.471782,0.504960,0.566427,0.691008
obEacy4Of68I,0.242208,0.829234,0.744090,0.627094,0.756269,0.699015,0.565864,0.463884,0.584879,0.295275,...,0.140506,-0.261833,0.269811,-0.023052,0.490709,0.684443,0.260870,0.373375,0.490076,0.617905
s7WzzDcmDOhF,0.275725,0.675102,0.702433,0.613503,0.804479,0.637771,0.286740,0.404671,0.331130,0.377236,...,-0.223803,0.010372,-0.154615,-0.602806,0.555110,-0.248578,0.254210,0.061103,0.132521,0.119855


In [6]:
X_aux_df.head()

,Basic_Demos_Enroll_Year_2016,PreInt_Demos_Fam_Child_Ethnicity_2.0,Barratt_Barratt_P2_Occ_25.0,SDQ_SDQ_Prosocial,PreInt_Demos_Fam_Child_Race_4.0,PreInt_Demos_Fam_Child_Ethnicity_1.0,Barratt_Barratt_P2_Occ_20.0,Basic_Demos_Enroll_Year_2017,Barratt_Barratt_P1_Edu_18.0,Barratt_Barratt_P2_Edu_15.0,...,Barratt_Barratt_P2_Occ_10.0,SDQ_SDQ_Emotional_Problems,ColorVision_CV_Score,APQ_P_APQ_P_OPD,PreInt_Demos_Fam_Child_Race_8.0,Barratt_Barratt_P1_Edu_9.0,Barratt_Barratt_P2_Occ_15.0,Basic_Demos_Enroll_Year_2019,APQ_P_APQ_P_PP,Basic_Demos_Study_Site_3
participant_id,,,,,,,,,,,,,,,,,,,,,
70z8Q2xdTXM3,0,0,0,0.900000,0,0,0,0,0,0,...,0,0.100000,1.0,0.476190,0,0,0,0,0.95,0
WHWymJu6zNZi,0,0,0,0.773816,0,1,0,0,0,0,...,0,0.233694,1.0,0.761905,1,0,1,0,0.85,0
4PAQp1M6EyAo,0,0,0,0.800000,0,0,1,0,1,0,...,0,0.400000,1.0,0.666667,0,0,0,1,0.95,0
obEacy4Of68I,0,0,0,1.000000,0,0,0,1,0,0,...,0,0.200000,1.0,0.666667,0,0,0,0,0.70,0
s7WzzDcmDOhF,0,1,0,0.600000,0,0,0,0,0,0,...,0,0.300000,1.0,0.523810,1,1,0,1,0.65,0


In [7]:
y_df.head()

,ADHD_Outcome,Sex_F
participant_id,,
70z8Q2xdTXM3,1,0
WHWymJu6zNZi,1,1
4PAQp1M6EyAo,1,1
obEacy4Of68I,1,1
s7WzzDcmDOhF,1,1


In [8]:
class Model(nn.Module):
    def __init__(self, input_dim, layer_dims, dropout, output_dim=4):
        super(Model, self).__init__()
        layers = []
        prev_dim = input_dim
        for dim in layer_dims:
            layers.append(nn.Linear(prev_dim, dim))
            layers.append(nn.ReLU())
            prev_dim = dim
        layers.append(nn.Dropout(dropout))
        layers.append(nn.Linear(prev_dim, output_dim))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

In [ ]:
X_fnc = np.array(X_fnc_df.values, dtype=np.float32)
X_aux = np.array(X_aux_df.values, dtype=np.float32)
y = np.array(y_df.values, dtype=np.float32)
y = np.array(y[:, 0] * 2 + y[:, 1], dtype=np.uint8)
X_fnc.shape, X_aux.shape, y.shape

((1213, 19900), (1213, 59), (1213,))

In [10]:
_, X_fnc, _, _ = train_test_split(
    X_fnc, y, test_size=0.2, random_state=42
)

In [11]:
_, X_aux, _, y = train_test_split(
    X_aux, y, test_size=0.2, random_state=42
)

In [12]:
X_fnc.shape, X_aux.shape, y.shape

((243, 19900), (243, 59), (243,))

In [13]:
fnc_model = Model(
    input_dim=X_fnc.shape[1], layer_dims=[256, 128, 64, 32], dropout=0.1
).to(device)
state_dict_fnc = torch.load('/Users/reza/School/2025/1-Spring/Advanced ML/Project/WiDS2025/results/unimodal/mlp_fnc/256-128-64-32-0.1.pt', map_location=device)
fnc_model.load_state_dict(state_dict_fnc)
fnc_model.eval()

/var/folders/bv/7h0f0hns2y72yqgg3ww8qkkm0000gn/T/ipykernel_9148/1194228735.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict_fnc = torch.load('/Users/reza/Schoo

Model(
  (layers): Sequential(
    (0): Linear(in_features=19900, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=32, out_features=4, bias=True)
  )
)

In [14]:
aux_model = Model(
    input_dim=X_aux.shape[1], layer_dims=[128, 64, 32, 16], dropout=0.3
).to(device)
state_dict_aux = torch.load('/Users/reza/School/2025/1-Spring/Advanced ML/Project/WiDS2025/results/unimodal/mlp_aux/128-64-32-16-0.3.pt', map_location=device)
aux_model.load_state_dict(state_dict_aux)
aux_model.eval()

/var/folders/bv/7h0f0hns2y72yqgg3ww8qkkm0000gn/T/ipykernel_9148/1268915761.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict_aux = torch.load('/Users/reza/Schoo

Model(
  (layers): Sequential(
    (0): Linear(in_features=59, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=16, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=16, out_features=4, bias=True)
  )
)

In [15]:
seed = 42  # Choose any fixed number
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # If using CUDA

In [16]:
out_fnc = fnc_model(torch.from_numpy(X_fnc).to(device)).cpu().detach()

In [17]:
out_fnc, out_fnc.shape

(tensor([[-1.5200e-01,  5.2935e-01,  3.0182e-01,  5.1287e-01],
         [ 1.5504e+00, -1.9756e-01,  1.2045e+00, -5.2781e-01],
         [ 3.1855e-01, -8.2087e-01,  1.2899e+00,  4.8965e-01],
         [ 1.5744e+00,  2.6517e-02,  1.3167e+00, -6.2637e-01],
         [ 3.8828e-01, -3.8306e-01,  8.5212e-01,  2.8978e-01],
         [-2.5981e-02,  8.0294e-01,  9.8526e-02,  5.9723e-01],
         [ 4.3512e-01, -8.3824e-02,  9.0112e-01,  1.2583e-01],
         [ 1.2331e+00, -5.8091e-01,  1.3836e+00, -2.0200e-01],
         [ 3.3639e-01, -5.0030e-02,  8.3226e-01,  4.9942e-01],
         [ 9.1100e-02, -4.0210e-01,  7.6854e-01,  6.2872e-01],
         [ 1.2495e+00, -1.9142e-01,  1.1377e+00, -4.9305e-01],
         [ 6.9722e-01, -3.9466e-01,  1.1891e+00,  1.4980e-01],
         [ 3.0092e-02,  1.0563e+00,  9.9675e-02,  1.3534e-01],
         [ 9.0198e-01,  5.8350e-02,  8.8327e-01, -1.2180e-01],
         [ 1.1926e+00, -5.7557e-01,  1.4714e+00, -2.0051e-01],
         [ 4.2308e-01, -1.0405e-01,  9.2615e-01,  3.945

In [18]:
out_aux = aux_model(torch.from_numpy(X_aux).to(device)).cpu().detach()

In [19]:
out_aux, out_aux.shape

(tensor([[-2.1737e+00, -1.4804e+00,  8.7426e-01,  8.8995e-01],
         [-7.9636e-01, -6.3315e-01,  2.8904e-01,  2.1445e-01],
         [-2.0110e+00, -1.4385e+00,  7.7422e-01,  7.9716e-01],
         [-4.0699e-01, -3.6599e-01,  1.4374e-01,  9.1331e-04],
         [-1.1566e+00, -8.7558e-01,  4.4949e-01,  4.1249e-01],
         [-6.6393e-01, -6.7758e-01,  1.8926e-01,  1.3996e-01],
         [ 2.0992e-01,  1.0515e-01, -5.0850e-01, -6.2763e-01],
         [-1.2130e+00, -1.0322e+00,  4.1344e-01,  4.5444e-01],
         [-9.8641e-01, -7.3817e-01,  3.9724e-01,  3.1739e-01],
         [-1.0469e+00, -9.8710e-01,  2.7063e-01,  3.3970e-01],
         [-8.9148e-02, -7.8080e-03, -5.5265e-01, -5.9126e-01],
         [-9.7482e-01, -7.6308e-01,  3.9115e-01,  3.3029e-01],
         [ 2.0350e-01,  2.9162e-01, -9.0455e-01, -9.6517e-01],
         [ 9.5453e-02,  2.9324e-01, -7.7039e-01, -8.0527e-01],
         [-8.7345e-01, -7.2048e-01,  3.0031e-01,  2.5654e-01],
         [-8.2108e-01, -7.8913e-01, -4.2721e-03,  1.111

In [20]:
pred_fnc = np.argmax(out_fnc, axis=1)
np.unique(pred_fnc, return_counts=True)

(array([0, 1, 2, 3]), array([ 53,  39, 102,  49]))

In [21]:
import torch.nn.functional as F

In [22]:
pred_aux = np.argmax(out_aux, axis=1)
np.unique(pred_aux, return_counts=True)

(array([0, 1, 2, 3]), array([27, 80, 89, 47]))

In [25]:
out_fnc_soft, out_aux_soft = F.softmax(out_fnc, dim=1), F.softmax(out_aux, dim=1)
print(out_fnc_soft.shape, out_aux_soft.shape)

torch.Size([243, 4]) torch.Size([243, 4])


In [32]:
out_avg = out_fnc_soft * 0.5 + out_aux_soft * 0.5
print(out_avg.shape)
pred = np.argmax(out_avg, axis=1)

torch.Size([243, 4])


In [33]:
def compute_leaderboard_f1_multiclass(y_true, y_pred):
    """
    Multiclass version of compute_leaderboard_f1_binary.
    Assumes class encoding:
        0 -> [ADHD=0, Sex_F=0]
        1 -> [ADHD=0, Sex_F=1]
        2 -> [ADHD=1, Sex_F=0]
        3 -> [ADHD=1, Sex_F=1]

    Returns:
    - average of two F1 scores:
        (1) ADHD F1 with extra weight on ADHD=1 & Sex_F=1
        (2) Sex_F F1 (unweighted)
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Decode back to binary labels
    true_adhd = y_true // 2  # 2 or 3 → 1
    true_sex_f = y_true % 2  # 1 or 3 → 1
    pred_adhd = y_pred // 2
    pred_sex_f = y_pred % 2

    # ADHD: apply weight=2 if true_adhd=1 and true_sex_f=1
    weights = np.where((true_adhd == 1) & (true_sex_f == 1), 2, 1)
    f1_adhd = f1_score(true_adhd, pred_adhd, sample_weight=weights, average="binary")
    f1_sex_f = f1_score(true_sex_f, pred_sex_f)

    return (f1_adhd + f1_sex_f) / 2

In [34]:
f1 = compute_leaderboard_f1_multiclass(y, pred)
print(f"F1 Score: {f1:.4f}")

F1 Score: 0.6993
